In [0]:
%fs ls /Workspace/Users/kanikr1810@gmail.com/

In [0]:
%pip install torch torchvision transformers opencv-python streamlit pillow scikit-learn

In [0]:
dbutils.library.restartPython()

In [0]:
%pip show torch torchvision streamlit

In [0]:

%pip install kaggle

In [0]:
dbutils.library.restartPython()

In [0]:


!mkdir -p /tmp/.kaggle
with open('/tmp/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"kanimozhikr","key":"b55c134d870dc6150c8f21e49f30fb35"}')




In [0]:

!kaggle datasets download -d anujms/car-damage-detection -p /tmp/
!unzip /tmp/car-damage-detection.zip -d /Workspace/Users/kanikr1810@gmail.com/


In [0]:
!ls /Workspace/Users/kanikr1810@gmail.com/data1a/training/00-damage/

In [0]:

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

# Path to dataset
data_dir = "/Workspace/Users/kanikr1810@gmail.com/data1a/"

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")


In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pretrained ResNet50 with updated syntax
model = resnet50(weights=ResNet50_Weights.DEFAULT)

# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer for binary classification (real vs fake)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# ✅ Save the trained model
torch.save(model.state_dict(), "/Workspace/Users/kanikr1810@gmail.com/fraud_detector.pth")
print("Model saved at /Workspace/Users/kanikr1810@gmail.com/")


In [0]:

%sh
cp /Workspace/Users/kanikr1810@gmail.com/fraud_detector.pth ./fraud_detector.pth



In [0]:
%pip install streamlit pyngrok torch torchvision pillow

In [0]:
print("Hi")

In [0]:
dbutils.library.restartPython()

In [0]:

%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# your model architecture
class FraudDetector(nn.Module):
    def __init__(self):
        super(FraudDetector, self).__init__()
        self.fc = nn.Linear(512, 2)  # Adjust based on your model

    def forward(self, x):
        return self.fc(x)

# Load model from current directory
model_path = "./fraud_detector.pth"
model = FraudDetector()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# Streamlit UI
st.title("Car Insurance Fraud Detection")
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    img_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=1)
        _, predicted = torch.max(output, 1)
        label = "Fraudulent" if predicted.item() == 1 else "Genuine"

    st.success(f"Prediction: {label}")


In [0]:

%sh
cp /tmp/fraud_detector.pth ./fraud_detector.pth


In [0]:


import base64

file_path = "/tmp/fraud_detector.pth"  # Change if needed
with open(file_path, "rb") as f:
    data = f.read()

b64 = base64.b64encode(data).decode()
download_link = f"data:application/octet-stream;base64,{b64}Click here to download fraud_detector.pth</a>"
displayHTML(download_link)
